In [1]:
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.linear_model import LogisticRegression
import pandas as pd
import numpy as np 
from sklearn.model_selection import KFold, train_test_split
import matplotlib.pyplot as plt, gc, os
import mlflow
import mlflow.sklearn
import mlflow.xgboost
import mlflow.lightgbm
from xgboost import XGBClassifier
import pickle
import warnings 
import xgboost as xgb

import lightgbm as lgb


import time

warnings.filterwarnings("ignore")


In [2]:
cat_cols = ['B_30', 'B_38', 'D_114', 'D_116', 'D_117', 'D_120', 'D_126', 'D_63', 'D_64', 'D_66', 'D_68']


In [3]:
labesl = pd.read_parquet("../data/train_iv_balanced.parquet")

In [5]:
FEATURES = labesl.columns.to_list()

In [7]:
FEATURES.remove("target")


In [3]:
#Woe_balanced dataframe
train = pd.read_parquet("../data/train_woe_balance_plus_original.parquet")


In [12]:
train= train.fillna(-127)

In [4]:
# train.drop('WOE_target',axis=1, inplace=True)
FEATURES = train.columns.to_list()
FEATURES.remove("target")
FEATURES = FEATURES[2:]
FEATURES

['P_2',
 'D_39',
 'B_1',
 'B_2',
 'R_1',
 'S_3',
 'D_41',
 'B_3',
 'D_42',
 'D_43',
 'D_44',
 'B_4',
 'D_45',
 'B_5',
 'R_2',
 'D_46',
 'D_47',
 'D_48',
 'D_49',
 'B_6',
 'B_7',
 'B_8',
 'D_50',
 'D_51',
 'B_9',
 'R_3',
 'D_52',
 'P_3',
 'B_10',
 'D_53',
 'S_5',
 'B_11',
 'S_6',
 'D_54',
 'R_4',
 'S_7',
 'B_12',
 'S_8',
 'D_55',
 'D_56',
 'B_13',
 'R_5',
 'D_58',
 'S_9',
 'B_14',
 'D_59',
 'D_60',
 'D_61',
 'B_15',
 'S_11',
 'D_62',
 'D_63',
 'D_64',
 'D_65',
 'B_16',
 'B_17',
 'B_18',
 'B_19',
 'D_66',
 'B_20',
 'D_68',
 'S_12',
 'R_6',
 'S_13',
 'B_21',
 'D_69',
 'B_22',
 'D_70',
 'D_71',
 'D_72',
 'S_15',
 'B_23',
 'D_73',
 'P_4',
 'D_74',
 'D_75',
 'D_76',
 'B_24',
 'R_7',
 'D_77',
 'B_25',
 'B_26',
 'D_78',
 'D_79',
 'R_8',
 'R_9',
 'S_16',
 'D_80',
 'R_10',
 'R_11',
 'B_27',
 'D_81',
 'D_82',
 'S_17',
 'R_12',
 'B_28',
 'R_13',
 'D_83',
 'R_14',
 'R_15',
 'D_84',
 'R_16',
 'B_29',
 'B_30',
 'S_18',
 'D_86',
 'D_87',
 'R_17',
 'R_18',
 'D_88',
 'B_31',
 'S_19',
 'R_19',
 'B_32',
 

In [11]:
#Competition metric
def amex_metric(y_true, y_pred):

    labels     = np.transpose(np.array([y_true, y_pred]))
    labels     = labels[labels[:, 1].argsort()[::-1]]
    weights    = np.where(labels[:,0]==0, 20, 1)
    cut_vals   = labels[np.cumsum(weights) <= int(0.04 * np.sum(weights))]
    top_four   = np.sum(cut_vals[:,0]) / np.sum(labels[:,0])

    gini = [0,0]
    for i in [1,0]:
        labels         = np.transpose(np.array([y_true, y_pred]))
        labels         = labels[labels[:, i].argsort()[::-1]]
        weight         = np.where(labels[:,0]==0, 20, 1)
        weight_random  = np.cumsum(weight / np.sum(weight))
        total_pos      = np.sum(labels[:, 0] *  weight)
        cum_pos_found  = np.cumsum(labels[:, 0] * weight)
        lorentz        = cum_pos_found / total_pos
        gini[i]        = np.sum((lorentz - weight_random) * weight)

    return 0.5 * (gini[1]/gini[0] + top_four)

In [12]:
# Model Parameters

# xgb_params = {
#     'max_depth':4, 
#     'learning_rate':0.05,
#     'subsample': 0.8,
#     'colsample_bytree':0.6,
#     'eval_metric': 'logloss',
#     'objective':'binary:logistic',
#     'tree_method':'gpu_hist',
#     'predictor':'gpu_predictor',
#     'random_state':42
# }

xgb_params = {
        'num_leaves': 10,
        'max_bin': 127,
        'min_data_in_leaf': 11,
        'learning_rate': 0.035,
        'bagging_fraction': 1.0, 
        'bagging_freq': 5, 
        'feature_fraction': 0.05,
        'lambda_l1': 4.972,
        'lambda_l2': 2.276,
        'min_gain_to_split': 0.65,
        'max_depth': 14,
        'save_binary': True,
        'seed': 1337,
        'feature_fraction_seed': 1337,
        'bagging_seed': 1337,
        'drop_seed': 1337,
        'data_random_seed': 1337,
        'objective': 'binary',
        'boosting_type': 'dart',
        'verbose': 1,
        'is_unbalance': True,
        'boost_from_average': False,
        'device': 'gpu',
        'tree_method':'gpu_hist',
        'predictor':'gpu_predictor',
        'gpu_platform_id': 0,
        'gpu_device_id': 0
        }

reg_log_params={
        'penalty': 'l2',
        'max_iter': 200,
        'warm_start': True,
        'n_jobs': 1,
            
                }
fores_params={
        'bootstrap': True,
        'criterion': 'gini',
        'max_depth': 20,
        'max_features': 'auto',
        'max_leaf_nodes': None,
        'min_impurity_decrease': 0.01,
        'min_samples_leaf': 1,
        'min_samples_split': 2,
        'min_weight_fraction_leaf': 0.0,
        'n_estimators': 10,
        'n_jobs': -1,
        'oob_score': False,
        'random_state': 42,
        'verbose': 0,
        'warm_start': False
            }

In [21]:
# Setting MLFlow
experiment_name = "RegLog, Forest raw dataset"
try:
    exp_id = mlflow.create_experiment(name=experiment_name)
except Exception as e:
    exp_id = mlflow.get_experiment_by_name(experiment_name).experiment_id 

## Reg Log && Random Forest

In [22]:
mlflow.autolog()

modelclasses = [
        #["reg_log",LogisticRegression,reg_log_params],
        ["forest", RandomForestClassifier, fores_params]
                ]

# TRAIN RANDOM SEED
SEED = 42

# FILL NAN VALUE
NAN_VALUE = -127 # will fit in int8

# FOLDS PER MODEL
FOLDS = 5


importances = []
oof = []
TRAIN_SUBSAMPLE = 1.0

skf = KFold(n_splits = FOLDS, shuffle=True, random_state=42)
with mlflow.start_run(experiment_id=exp_id):
        for fold, (train_idx, valid_idx) in enumerate(skf.split(train, train.target)):
                

                if TRAIN_SUBSAMPLE<1.0:
                        np.random.seed(SEED)
                        train_idx = np.random.choice(train_idx, 
                                        int(len(train_idx)*TRAIN_SUBSAMPLE), replace=False)
                        np.random.seed(None)
                
                X_train = train.loc[train_idx, FEATURES]
                y_train = train.loc[train_idx, 'target']
                X_valid = train.loc[valid_idx, FEATURES]
                y_valid = train.loc[valid_idx, 'target']

                for modelname, Model, param_list in modelclasses:
                        print('#'*25)
                        print('### Fold',fold+1)
                        print('### Train size',len(train_idx),'Valid size',len(valid_idx))
                        print(f'### Training model {modelname.upper()}')
                        print(f'### Training with {int(TRAIN_SUBSAMPLE*100)}% fold data...')
                        print('#'*25)

                        model = Model(**param_list)
                        #print(model)
                        model.fit(X_train,y_train)

                        pickle.dump(model, open(f'../models/{modelname}_fold{fold}.pkl','wb'))

                        oof_preds = model.predict(X_valid)
                        acc = amex_metric(y_valid.values, oof_preds)
                        print("Kaggle Metric=", acc,'\n')
                        mlflow.log_metric(f"Kaggle Metric for {modelname}", acc)
                        mlflow.sklearn.log_model(model, f"{Model}")
                        print("Model saved in run %s" % mlflow.active_run().info.run_uuid)
                        
                        mlflow.sklearn.log_model(model, f"{Model}")
                        df = train.loc[valid_idx, ['customer_ID', 'target']].copy()
                        df['oof_pred']= oof_preds
                        df['model_name'] = modelname
                        oof.append(df)

                        del df, model

                del X_train, y_train
                del X_valid, y_valid

        
print('#'*25)
oof = pd.concat(oof, axis=0, ignore_index=True).set_index('customer_ID')
for n in range(len(modelclasses)):
    target = oof.loc[oof['model_name'] ==modelclasses[n][0], ['target'] ].reset_index()
    preds = oof.loc[oof['model_name'] == modelclasses[n][0], ['oof_pred']].reset_index()
    acc= amex_metric(target.target.values, preds.oof_pred.values)
    print(f'OVERAL CV Kaggle Metric for {modelclasses[n][0]} = {acc}')



2022/09/29 16:08:29 INFO mlflow.tracking.fluent: Autologging successfully enabled for lightgbm.
2022/09/29 16:08:29 INFO mlflow.tracking.fluent: Autologging successfully enabled for sklearn.
2022/09/29 16:08:29 INFO mlflow.tracking.fluent: Autologging successfully enabled for xgboost.


#########################
### Fold 1
### Train size 4425160 Valid size 1106291
### Training model FOREST
### Training with 100% fold data...
#########################
Kaggle Metric= 0.3829176989456994 

Model saved in run a5a0f74153e14cf39db43780d2a7f7d8
#########################
### Fold 2
### Train size 4425161 Valid size 1106290
### Training model FOREST
### Training with 100% fold data...
#########################
Kaggle Metric= 0.3833680140472879 

Model saved in run a5a0f74153e14cf39db43780d2a7f7d8
#########################
### Fold 3
### Train size 4425161 Valid size 1106290
### Training model FOREST
### Training with 100% fold data...
#########################
Kaggle Metric= 0.38209591391022013 

Model saved in run a5a0f74153e14cf39db43780d2a7f7d8
#########################
### Fold 4
### Train size 4425161 Valid size 1106290
### Training model FOREST
### Training with 100% fold data...
#########################
Kaggle Metric= 0.3827378199067674 

Model saved in run a5a0f74153e1

## XgBoost

In [13]:
# Setting MLFlow
experiment_name = "XGBoost - WoE Balanced + IV Balanced"
try:
    exp_id = mlflow.create_experiment(name=experiment_name)
except Exception as e:
    exp_id = mlflow.get_experiment_by_name(experiment_name).experiment_id 

In [8]:
mlflow.autolog()
importances = []
oof = []
TRAIN_SUBSAMPLE = 1.0

skf = KFold(n_splits = 5, shuffle=True, random_state=42)

for fold, (train_idx, valid_idx) in enumerate(skf.split(train, train.target)):

    if TRAIN_SUBSAMPLE<1.0:
        np.random.seed(42)
        train_idx = np.random.choice(train_idx, 
                       int(len(train_idx)*TRAIN_SUBSAMPLE), replace=False)
        np.random.seed(None)
        
    print('#'*25)
    print('### Fold',fold+1)
    print('### Train size',len(train_idx),'Valid size',len(valid_idx))
    print(f'### Training with {int(TRAIN_SUBSAMPLE*100)}% fold data...')
    print('#'*25)

    X_train = train.loc[train_idx, FEATURES]
    y_train = train.loc[train_idx, 'target']
    X_valid = train.loc[valid_idx, FEATURES]
    y_valid = train.loc[valid_idx, 'target']

    # batch_size = 200000
    # model = None
    # for start in range(0, len(X_train), batch_size):

    dtrain=xgb.DMatrix(X_train, 
                        y_train)
    del X_train, y_train
    gc.collect()
    d_valid = xgb.DMatrix(X_valid, 
                        y_valid)
    del X_valid
    gc.collect()
    model = xgb.train(
    **xgb_params,
    dtrain=dtrain,
    evals=[(dtrain, 'train'), (d_valid, 'test')],
    num_boost_round= 9999,
    early_stopping_rounds = 100,
    verbose_eval= 100
                                
    )

    model.save_model(f'../models/XGB_V_fold{fold}.xgb')
    mlflow.xgboost.log_model(model, "XGBClassifier")

    dd = model.get_score(importance_type='weight')
    df= pd.DataFrame({'feature':dd.keys(), f'importance_{fold}':dd.values()})
    importances.append(df)
    
    oof_preds = model.predict(d_valid)
    acc = amex_metric(y_valid.values, oof_preds)
    mlflow.log_metric("Kaggle Metric for XGBClassifier", acc)

    print("Kaggle Metric=", acc,'\n')

    df = train.loc[valid_idx, ['customer_ID', 'target']].copy()
    df['oof_pred']= oof_preds
    oof.append(df)

    del   dd, df
    del  d_valid, model

print('#'*25)
oof = pd.concat(oof, axis=0, ignore_index=True).set_index('customer_ID')
acc= amex_metric(oof.target.values, oof.oof_pred.values)
print('OVERAL CV Kaggle Metric = ', acc)


2022/09/29 16:17:55 INFO mlflow.tracking.fluent: Autologging successfully enabled for sklearn.
2022/09/29 16:17:55 INFO mlflow.tracking.fluent: Autologging successfully enabled for lightgbm.
2022/09/29 16:17:55 INFO mlflow.tracking.fluent: Autologging successfully enabled for xgboost.


#########################
### Fold 1
### Train size 4425160 Valid size 1106291
### Training with 100% fold data...
#########################


2022/09/29 16:18:11 INFO mlflow.utils.autologging_utils: Created MLflow autologging run with ID 'b17463ab9c114583997aa80e2db8573d', which will track hyperparameters, performance metrics, model artifacts, and lineage information for the current xgboost workflow


[0]	train-logloss:0.66626	test-logloss:0.66628
[100]	train-logloss:0.28711	test-logloss:0.28713
[200]	train-logloss:0.27706	test-logloss:0.27715
[300]	train-logloss:0.27303	test-logloss:0.27325
[400]	train-logloss:0.27069	test-logloss:0.27099
[500]	train-logloss:0.26908	test-logloss:0.26946
[600]	train-logloss:0.26781	test-logloss:0.26830
[700]	train-logloss:0.26681	test-logloss:0.26741
[800]	train-logloss:0.26595	test-logloss:0.26663
[900]	train-logloss:0.26518	test-logloss:0.26595
[1000]	train-logloss:0.26452	test-logloss:0.26539
[1100]	train-logloss:0.26392	test-logloss:0.26487
[1200]	train-logloss:0.26337	test-logloss:0.26440
[1300]	train-logloss:0.26287	test-logloss:0.26400
[1400]	train-logloss:0.26239	test-logloss:0.26361
[1500]	train-logloss:0.26195	test-logloss:0.26325
[1600]	train-logloss:0.26152	test-logloss:0.26290
[1700]	train-logloss:0.26111	test-logloss:0.26259
[1800]	train-logloss:0.26070	test-logloss:0.26228
[1900]	train-logloss:0.26032	test-logloss:0.26198
[2000]	train

# LGBM

In [16]:
# Setting MLFlow
experiment_name = "LightGBM"
try:
    exp_id = mlflow.create_experiment(name=experiment_name)
except Exception as e:
    exp_id = mlflow.get_experiment_by_name(experiment_name).experiment_id 

In [ ]:
def lgb_amex_metric(y_pred, y_true):
    y_true = y_true.get_label()
    return 'amex_metric', amex_metric(y_true, y_pred), True


params = {
    'objective': 'binary',
    'metric': 'binary_logloss',
    'boosting': 'dart',
    'seed': 42,
    'num_leaves': 100,
    'learning_rate': 0.01,
    'feature_fraction': 0.20,
    'bagging_freq': 10,
    'bagging_fraction': 0.50,
    'n_jobs': -1,
    'lambda_l2': 2,
    'min_data_in_leaf': 40,
    'device_type': 'gpu',
    'max_bin': 64,

    }
# Create a numpy array to store test predictions
#test_predictions = np.zeros(len(test))
# Create a numpy array to store out of folds predictions
oof_predictions = np.zeros(len(train))
skf = KFold(n_splits = 5, shuffle=True, random_state=42)
for fold, (train_idx, valid_idx) in enumerate(skf.split(train, train.target)):
    print(' ')
    print('-'*50)
    print(f'Training fold {fold} with {len(FEATURES)} features...')
    x_train, x_val = train[FEATURES].iloc[train_idx], train[FEATURES].iloc[valid_idx]
    y_train, y_val = train['target'].iloc[train_idx], train['target'].iloc[valid_idx]
    lgb_train = lgb.Dataset(x_train, y_train)
    lgb_valid = lgb.Dataset(x_val, y_val)
    model = lgb.train(
        params = params,
        train_set = lgb_train,
        num_boost_round = 10500,
        valid_sets = [lgb_train, lgb_valid],
        early_stopping_rounds = 1500,
        verbose_eval = 500,
        feval = lgb_amex_metric
        )
    # Save best model
    pickle.dump(model, open(f'../models/LGBM_fold{fold}.pkl','wb'))
    # Predict validation
    val_pred = model.predict(x_val)
    # Add to out of folds array
    oof_predictions[valid_idx] = val_pred
    # Predict the test set
    #test_pred = model.predict(test[FEATURES])
    #test_predictions += test_pred / 5
    # Compute fold metric
    score = amex_metric(y_val, val_pred)
    print(f'Our fold {fold} CV score is {score}')
    mlflow.log_metric("Kaggle Metric for LightGbm", acc)
    mlflow.lightgbm.log_model(model, f"{Model}")
    del x_train, x_val, y_train, y_val, lgb_train, lgb_valid
    gc.collect()
# Compute out of folds metric
score = amex_metric(train[target], oof_predictions)
print(f'Our out of folds CV score is {score}')
# Create a dataframe to store out of folds predictions
oof_df = pd.DataFrame({'customer_ID': train['customer_ID'], 'target': train[target], 'prediction': oof_predictions})
oof_df.to_csv(f'/content/drive/MyDrive/Amex/OOF/oof_lgbm_dart_baseline_5fold_seed42.csv', index = False)
# Create a dataframe to store test prediction
# test_df = pd.DataFrame({'customer_ID': test['customer_ID'], 'prediction': test_predictions})
# test_df.to_csv(f'/content/drive/MyDrive/Amex/Predictions/test_lgbm_dart_baseline_fold_5_seed42.csv', index = False)

